In [14]:
from numba import jit, cuda 
from scipy.signal import convolve2d
from scipy.integrate import ode
import numpy as np
from PIL import Image as img

In [26]:
# hàm trạng thái
def hamTrangThai(t, x, uB1, uB2, A1, A2, I, n, m):
    x = x.reshape(n, m)
    dx = uB1 + convolve2d(hamDauRa(x), A1, 'same') + uB2 + tichChapBacHai(hamDauRa(x), A2) + I
    #dx = convolve2d(u, B1, 'same') + convolve2d(hamDauRa(x), A1, 'same') + tichChapBacHai(u, B2) + tichChapBacHai(hamDauRa(x), A2) + I
    return dx.reshape(n*m)

# Hàm tính tích chập bậc hai chạy trên GPU thông qua thư viện Numba (vì chạy trên CPU quá chậm)
@jit(target_backend='cuda')
#def tichChapBacHai_numba(y, A):
def tichChapBacHai(y, A):
    m, n = y.shape
    ketQua = np.zeros(y.shape) # tạo ma trận 0 có kích thước bằng ma trận y để lưu kết quả
    for i in range(m):
        for j in range(n):
            # xét từng điểm trên ma trận y kích thước m, n
            for k in range(3):#0.. 2
                for l in range(3):#0.. 2
                    # xét từng ma trận trong bộ A21 đến A29 (trong code là A[0] đến A[8])
                    x = k*3+l # ma trận A[x] đang được xét
                    for r in range(-1,2):#-1.. 1
                        for s in range(-1,2):#-1.. 1
                            # nhân từng số của ma trận láng giềng y(i,j;r,s) với ma trận A[x]
                            if(0<=i+r<m and 0<=j+s<n and 0<=i+1-k<m and 0<=j+1-l<n):
                                ketQua[i][j]+=A[x][r][s]*y[i+1-k][j+1-l]*y[i+r][j+s]
    return ketQua

# hàm đầu ra
def hamDauRa(x):
    return 0.5 * (np.abs(x + 1) - np.abs(x - 1))

# Tạo ma trận đối xứng từ 5 trọng số (hàm này chỉ dành riêng cho ma trận đối xứng 3x3)
def taoMaTranDoiXung(arr1):#giả sử cho mảng [1,2,3,4,5]
    arr2 = arr1[-2::-1].copy()#tạo mảng phụ [4,3,2,1]
    #arr2[0], arr2[2] = arr2[2], arr2[0]#nếu có thì sẽ đối xứng theo đường chéo phụ, nếu không thì sẽ là đối xứng tâm
    #print(f'arr1 = {arr1}, arr2 = {arr2}')
    return np.append(arr1,arr2).reshape(h,h)#kết hợp hai mảng rồi chuyển về dạng ma trận 3x3

# Trong công thức biến đổi từ mạng Nơ-ron phẩn hồi sang truyền thẳng, trọng số và đầu vào , đầu ra phải được nhân với nhau theo thứ tự nhất định (xem lại công thứ)
# hàm sau đây sẽ sắp xếp lại các số trọng bộ ma trận trọng số để tương ứng với công thức.
def taoMaTranDoiXungPerceptron(arr1):
    return taoMaTranDoiXung(np.array([arr1[2],arr1[1],arr1[3],arr1[0],arr1[4]]))

# Hàm tạo số ngẫu nhiên từ -9.99 đến 9.99
def ngauNhien():
    return np.random.randint(-999,1000)/100.

# Hàm tính sai số
def saiSo(d,y):
    return np.sum(0.5*(d-y)**2)

def loadSample(path):
    return np.array(img.open(path).convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0

# Bán kính lân cận của nơ ron tế bào
r = 1

# kích thước của ma trận điều khiển và ma trận phản hồi (hai ma trận này luôn vuông)
h = 2*r + 1 # h = 3

class CNN:

    slts = 101 # số lượng trọng số trong quần thể

    # Quần thể trọng số (gồm 11 số)
    #quanThe = np.array([ngauNhien() for i in range(slts)])
    quanThe = np.array([0.,0.,0.,0.,2.]*10+[0.,0.,0.,0.,0.]*10+[1.])
    #quanThe = np.array([4.]*101)

    # ma trận ảnh đầu vào u
    # u = np.array(img.open('images/u1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0
    u = [
        loadSample('images/u1_1.png'),
        loadSample('images/u1_2.png'),
        loadSample('images/u1_3.png'),
        loadSample('images/u1_4.png'),
        loadSample('images/u1_5.png'),
    ]

    # kích thước ma trận đầu vào, ma trận đầu ra và ma trận trạng thái
    m, n = u[0].shape
    print(m,n)

    # ma trận đầu ra mong muốn d
    # d = np.array(img.open('images/d1.png').convert('RGB'), dtype=float)[:,:,0]/255.0*2.0-1.0
    d = [
        loadSample('images/d1_1.png'),
        loadSample('images/d1_2.png'),
        loadSample('images/d1_3.png'),
        loadSample('images/d1_4.png'),
        loadSample('images/d1_5.png'),
    ]

    # hai giá trị ban đầu cho ptvp gồm x0 = [ma trận toàn số 0] với t0 = 0.0 và t1 để tính đầu ra x1 (t1 != t0)
    x0, t0, t1 = [i.copy() for i in u], 0.0, 0.02

    # ma trận đầu ra y (y được tính theo x)
    y = [hamDauRa(i) for i in x0]

    # Tốc độ học của thuật toán
    alpha = [0.02]*len(u)

    alpha0 = 0.005

    E0 = 10.

    E = [1000]*len(u)

cnn = CNN()

# In ra bộ ma trận trọng số:
def print_maTranTrongSo():
    print(f'A1 = {taoMaTranDoiXung(cnn.quanThe[:5])}')
    for i in range(9):
        print(f'A2{i+1} = {taoMaTranDoiXung(cnn.quanThe[i*5+5:i*5+10])}')
    print(f'B1 = {taoMaTranDoiXung(cnn.quanThe[50:55])}')
    for i in range(9):
        print(f'B2{i+1} = {taoMaTranDoiXung(cnn.quanThe[i*5+55:i*5+60])}')
    print(f'I = {cnn.quanThe[100]}')

# tạo thêm biên gồm toàn số 0 bao quanh ma trận a để tránh tràn biên
def them_bien(a):
    b = np.zeros(shape=[a.shape[0]+2,a.shape[1]+2])
    for i in range(1,a.shape[0]+1):
        for j in range(1,a.shape[1]+1):
            b[i][j] = a[i-1][j-1]
    return b

# Hàm để lấy trọng số từ file trongSoBac2ToiUu rồi lưu vào cnn.quanThe
def getQuanThe(i):# lấy bộ trọng số thứ i
    A1 = np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/A1.txt')
    A2 = []
    for j in range(1,10):
        A2.append(np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/A2{j}.txt'))
    B1 = np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/B1.txt')
    B2 = []
    for j in range(1,10):
        B2.append(np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/B2{j}.txt'))
    I = np.loadtxt(f'TrongSoBac2ToiUu/TrongSoBac2_{i}/I.txt')
    qt = [A1[0][0],A1[0][1],A1[0][2],A1[1][0],A1[1][1]]
    for A2x in A2:
        qt += [A2x[0][0],A2x[0][1],A2x[0][2],A2x[1][0],A2x[1][1]]
    qt += [B1[0][0],B1[0][1],B1[0][2],B1[1][0],B1[1][1]]
    for B2x in B2:
        qt += [B2x[0][0],B2x[0][1],B2x[0][2],B2x[1][0],B2x[1][1]]
    qt += [I]
    cnn.quanThe = np.array(qt)

# tính x, tính y, tính wij và cập trọng số
def capNhatTrongSo():
    for t in range(len(cnn.u)):
        cnn.alpha[t]=cnn.alpha0*(cnn.E[t]-cnn.E0)/cnn.m/cnn.n if cnn.E[t] > cnn.E0 else 0.
        for i in range(1,cnn.m):
            for j in range(1,cnn.n):
                # cập nhật trọng số
                y = them_bien(cnn.y[t]) # thêm biên 0 cho y để tránh tràn biên
                u = them_bien(cnn.u[t]) # thêm biên 0 cho u để tránh tràn biên  

                Ys = [
                    y[i-1][j] + y[i+1][j], 
                    y[i][j-1] + y[i][j+1],
                    y[i-1][j-1] + y[i+1][j+1],
                    y[i-1][j+1] + y[i+1][j-1],
                    y[i][j]
                ]
                for k in range(0,9):
                    Ys += [
                        (y[i-1][j] + y[i+1][j]) * y[k//3][k%3],
                        (y[i][j-1] + y[i][j+1]) * y[k//3][k%3],
                        (y[i-1][j-1] + y[i+1][j+1]) * y[k//3][k%3],
                        (y[i-1][j+1] + y[i+1][j-1]) * y[k//3][k%3],
                        y[i][j] * y[k//3][k%3]
                    ]
                Ys += [
                    u[i-1][j] + u[i+1][j], 
                    u[i][j-1] + u[i][j+1],
                    u[i-1][j-1] + u[i+1][j+1],
                    u[i-1][j+1] + u[i+1][j-1],
                    u[i][j]
                ]
                for k in range(0,9):
                    Ys += [
                        (u[i-1][j] + u[i+1][j]) * u[k//3][k%3],
                        (u[i][j-1] + u[i][j+1]) * u[k//3][k%3],
                        (u[i-1][j-1] + u[i+1][j+1]) * u[k//3][k%3],
                        (u[i-1][j+1] + u[i+1][j-1]) * u[k//3][k%3],
                        u[i][j] * u[k//3][k%3]
                    ]
                Ys += [1]
                Ys = np.array(Ys)
                norm_n = np.sum(Ys**2)
                wij = cnn.alpha[t] * (cnn.d[t][i-1][j-1] - y[i][j]) / norm_n
                if(wij != 0.):
                    for ts in range(cnn.slts):
                        cnn.quanThe[ts] += wij * Ys[ts]

                # tính toán lại sai số
                ptvp[t].set_initial_value(cnn.x0[t].flatten(), 0.)
                A1 = taoMaTranDoiXungPerceptron(cnn.quanThe[:5])
                A2 = []
                for c in range(1,10):
                    A2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*c:5*c+5]))
                B1 = taoMaTranDoiXungPerceptron(cnn.quanThe[50:55])
                B2 = []
                for c in range(1,10):
                    B2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*c+50:5*c+55]))
                I = cnn.quanThe[100]
                uB1 = convolve2d(cnn.u[t], B1, 'same')
                uB2 = tichChapBacHai(cnn.u[t], B2)
                ptvp[t].set_f_params(uB1, uB2, A1, A2, I, cnn.n, cnn.m)            
                x = ptvp[t].integrate(cnn.t1).reshape(cnn.n, cnn.m)
                cnn.y[t] = np.sign(hamDauRa(x))

ptvp = [ode(hamTrangThai).set_integrator('vode')]*len(cnn.u) # thuật toán được sử dụng để giải ptvp[i], đọc thêm trong tài liệu

getQuanThe(22)
print(f'slts: {cnn.slts}')
print_maTranTrongSo()
for i in range(len(cnn.u)):
    print(f'u[{i}] = \n{cnn.u[i]}')
    print(f'd[{i}] = \n{cnn.d[i]}')

A1 = taoMaTranDoiXungPerceptron(cnn.quanThe[:5])
A2 = []
for j in range(1,10):
    A2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j:5*j+5]))
B1 = taoMaTranDoiXungPerceptron(cnn.quanThe[50:55])
B2 = []
for j in range(1,10):
    B2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j+50:5*j+55]))
I = cnn.quanThe[100]

for i in range(len(cnn.u)):
    ptvp[i].set_initial_value(cnn.x0[i].flatten(), 0.) # lệnh flatten dùng để biến đổi ma trận về dạng mảng (đầu vào của ode phải là một số hoặc mảng)
    uB1 = convolve2d(cnn.u[i], B1, 'same')
    uB2 = tichChapBacHai(cnn.u[i], B2)
    ptvp[i].set_f_params(uB1, uB2, A1, A2, I, cnn.n, cnn.m)    
    assert ptvp[i].successful()
    x = ptvp[i].integrate(cnn.t1).reshape(cnn.n, cnn.m)
    y = hamDauRa(x)
    cnn.y[i] = np.sign(y)
    print(f'y = \n{cnn.y[i]}')
    cnn.E[i] = saiSo(cnn.d[i],cnn.y[i])
    print(f'E = \n{cnn.E[i]}')
print(f'sai số: {cnn.E}. Tổng: {sum(cnn.E)}')

8 8
slts: 101
A1 = [[ 0.    0.    0.  ]
 [-0.08  9.86 -0.08]
 [ 0.    0.    0.  ]]
A21 = [[0. 0. 0.]
 [0. 2. 0.]
 [0. 0. 0.]]
A22 = [[-0.25 -0.07  0.  ]
 [ 0.    1.57  0.  ]
 [ 0.   -0.07 -0.25]]
A23 = [[-0.03  0.    0.  ]
 [ 0.    1.24  0.  ]
 [ 0.    0.   -0.03]]
A24 = [[0.   0.34 0.19]
 [0.62 2.   0.62]
 [0.19 0.34 0.  ]]
A25 = [[-2.45  1.23  0.  ]
 [ 0.    2.    0.  ]
 [ 0.    1.23 -2.45]]
A26 = [[0.   0.21 0.  ]
 [0.   1.35 0.  ]
 [0.   0.21 0.  ]]
A27 = [[0.   0.   0.  ]
 [0.   2.17 0.  ]
 [0.   0.   0.  ]]
A28 = [[0.   0.57 0.  ]
 [0.36 1.55 0.36]
 [0.   0.57 0.  ]]
A29 = [[2.03 0.   0.  ]
 [0.   1.28 0.  ]
 [0.   0.   2.03]]
B1 = [[ 0.84  0.    0.  ]
 [ 0.   -5.4   0.  ]
 [ 0.    0.    0.84]]
B21 = [[ 0.    0.   -0.67]
 [ 0.    0.    0.  ]
 [-0.67  0.    0.  ]]
B22 = [[0.   0.   0.  ]
 [0.   0.02 0.  ]
 [0.   0.   0.  ]]
B23 = [[0.   0.   1.49]
 [0.   0.   0.  ]
 [1.49 0.   0.  ]]
B24 = [[0.  0.  0. ]
 [0.  0.3 0. ]
 [0.  0.  0. ]]
B25 = [[ 0.13  0.    0.86]
 [-0.29  0.   -0.29

d:\anaconda\anaconda3\lib\site-packages\numba\core\ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'A' of function 'tichChapBacHai'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "C:\Users\phamd\AppData\Local\Temp\ipykernel_9524\2359636939.py", line 10:
@jit(target_backend='cuda')
#def tichChapBacHai_numba(y, A):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


y = 
[[ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1.  1. -1.  1.  1. -1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1. -1.]
 [ 1.  1. -1. -1. -1.  1.  1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]]
E = 
26.0
y = 
[[-1. -1. -1. -1.  1.  1.  1.  1.]
 [-1. -1. -1. -1.  1. -1. -1. -1.]
 [-1. -1. -1. -1.  1. -1. -1. -1.]
 [-1. -1. -1. -1.  1. -1. -1. -1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]]
E = 
20.0
y = 
[[ 1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1. -1. -1.  1.  1.]
 [ 1.  1. -1. -1. -1. -1.  1.  1.]
 [ 1. -1. -1. -1. -1. -1. -1.  1.]
 [ 1. -1. -1. -1. -1. -1. -1.  1.]
 [ 1. -1. -1. -1. -1. -1.  1.  1.]
 [ 1.  1.  1. -1. -1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.]]
E = 
24.0
y = 
[[ 1.  1. -1. -1. -1. -1.  1.  1.]
 [-1. -1. -1. -1. -1. -1. -1.  1.]
 [-1. -1. -1. -1. -1. -1. -1. -1.]
 [

In [27]:
print('Bộ trọng số ban đầu: ')
print_maTranTrongSo()
print(f'Tổng sai số ban đầu: {sum(cnn.E)}')
# vòng lặp lớn
loop = 10000 #số lần chạy thử tối đa
count = 0 # đếm số lần chạy
while(True):
    # kiểm tra ptvp ODE
    if([i.successful() for i in ptvp].count(False)):
        print('Lỗi phương trình vi phân')
        break

    # đếm số lần chạy
    if(count>loop):
        print('Đạt đến số vòng lặp tối đa')
        break
    else:
        print(f'Lần chạy thứ {count+1}')
        
    capNhatTrongSo()

    count+=1

    cnn.E = [saiSo(cnn.d[i],cnn.y[i]) for i in range(len(cnn.u))]
    if(max(cnn.E) <= cnn.E0):
        print('Đạt đến mức sai số cho phép')
        break

    # print_maTranTrongSo()
    print(f'Tốc độ học: {cnn.alpha}')
    print(f'Sai số: {cnn.E}. Tổng sai số: {sum(cnn.E)}')

print('Bộ trọng sau khi tối ưu: ')
print_maTranTrongSo()
print(f'Sai số: {cnn.E}. Tổng sai số: {sum(cnn.E)}')

Bộ trọng số ban đầu: 
A1 = [[ 0.    0.    0.  ]
 [-0.08  9.86 -0.08]
 [ 0.    0.    0.  ]]
A21 = [[0. 0. 0.]
 [0. 2. 0.]
 [0. 0. 0.]]
A22 = [[-0.25 -0.07  0.  ]
 [ 0.    1.57  0.  ]
 [ 0.   -0.07 -0.25]]
A23 = [[-0.03  0.    0.  ]
 [ 0.    1.24  0.  ]
 [ 0.    0.   -0.03]]
A24 = [[0.   0.34 0.19]
 [0.62 2.   0.62]
 [0.19 0.34 0.  ]]
A25 = [[-2.45  1.23  0.  ]
 [ 0.    2.    0.  ]
 [ 0.    1.23 -2.45]]
A26 = [[0.   0.21 0.  ]
 [0.   1.35 0.  ]
 [0.   0.21 0.  ]]
A27 = [[0.   0.   0.  ]
 [0.   2.17 0.  ]
 [0.   0.   0.  ]]
A28 = [[0.   0.57 0.  ]
 [0.36 1.55 0.36]
 [0.   0.57 0.  ]]
A29 = [[2.03 0.   0.  ]
 [0.   1.28 0.  ]
 [0.   0.   2.03]]
B1 = [[ 0.84  0.    0.  ]
 [ 0.   -5.4   0.  ]
 [ 0.    0.    0.84]]
B21 = [[ 0.    0.   -0.67]
 [ 0.    0.    0.  ]
 [-0.67  0.    0.  ]]
B22 = [[0.   0.   0.  ]
 [0.   0.02 0.  ]
 [0.   0.   0.  ]]
B23 = [[0.   0.   1.49]
 [0.   0.   0.  ]
 [1.49 0.   0.  ]]
B24 = [[0.  0.  0. ]
 [0.  0.3 0. ]
 [0.  0.  0. ]]
B25 = [[ 0.13  0.    0.86]
 [-0.29  0.

KeyboardInterrupt: 

In [23]:
# Lưu lại bộ trọng số
np.savetxt('TrongSoBac2_Per/A1.txt',taoMaTranDoiXung(cnn.quanThe[:5]))
for i in range(9):
    np.savetxt(f'TrongSoBac2_Per/A2{i+1}.txt',taoMaTranDoiXung(cnn.quanThe[i*5+5:i*5+10]))
np.savetxt('TrongSoBac2/B1.txt',taoMaTranDoiXung(cnn.quanThe[50:55]))
for i in range(9):
    np.savetxt(f'TrongSoBac2_Per/B2{i+1}.txt',taoMaTranDoiXung(cnn.quanThe[i*5+55:i*5+60]))
np.savetxt('TrongSoBac2_Per/I.txt',cnn.quanThe[100:101])
print('Đã lưu')

Đã lưu


In [24]:
print(f'slts: {cnn.slts}')
print_maTranTrongSo()
for i in range(len(cnn.u)):
    print(f'u[{i}] = \n{cnn.u[i]}')
    print(f'd[{i}] = \n{cnn.d[i]}')

A1 = taoMaTranDoiXungPerceptron(cnn.quanThe[:5])
A2 = []
for j in range(1,10):
    A2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j:5*j+5]))
B1 = taoMaTranDoiXungPerceptron(cnn.quanThe[50:55])
B2 = []
for j in range(1,10):
    B2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j+50:5*j+55]))
I = cnn.quanThe[100]

for i in range(len(cnn.u)):
    ptvp[i].set_initial_value(cnn.x0[i].flatten(), 0.) # lệnh flatten dùng để biến đổi ma trận về dạng mảng (đầu vào của ode phải là một số hoặc mảng)
    uB1 = convolve2d(cnn.u[i], B1, 'same')
    uB2 = tichChapBacHai(cnn.u[i], B2)
    ptvp[i].set_f_params(uB1, uB2, A1, A2, I, cnn.n, cnn.m)    
    assert ptvp[i].successful()
    x = ptvp[i].integrate(cnn.t1).reshape(cnn.n, cnn.m)
    y = hamDauRa(x)
    cnn.y[i] = np.sign(y)
    print(f'y = \n{cnn.y[i]}')
    cnn.E[i] = saiSo(cnn.d[i],cnn.y[i])
    print(f'E = \n{cnn.E[i]}')
print(f'sai số: {cnn.E}. Tổng: {sum(cnn.E)}')

slts: 101
A1 = [[-1.06871021 -1.24461204 -0.68639752]
 [-1.06048492  8.98384573 -1.06048492]
 [-0.68639752 -1.24461204 -1.06871021]]
A21 = [[0. 0. 0.]
 [0. 2. 0.]
 [0. 0. 0.]]
A22 = [[-0.25 -0.07  0.  ]
 [ 0.    1.57  0.  ]
 [ 0.   -0.07 -0.25]]
A23 = [[-0.03  0.    0.  ]
 [ 0.    1.24  0.  ]
 [ 0.    0.   -0.03]]
A24 = [[0.   0.34 0.19]
 [0.62 2.   0.62]
 [0.19 0.34 0.  ]]
A25 = [[-2.50961624  1.34047392  0.26755571]
 [-0.0167776   2.03544845 -0.0167776 ]
 [ 0.26755571  1.34047392 -2.50961624]]
A26 = [[-0.05961624  0.32047392  0.26755571]
 [-0.0167776   1.38544845 -0.0167776 ]
 [ 0.26755571  0.32047392 -0.05961624]]
A27 = [[0.   0.   0.  ]
 [0.   2.17 0.  ]
 [0.   0.   0.  ]]
A28 = [[0.6366398  1.3786758  0.94938287]
 [0.98679411 1.98402375 0.98679411]
 [0.94938287 1.3786758  0.6366398 ]]
A29 = [[2.71146817 0.50696582 1.05106555]
 [0.71700912 1.33023899 0.71700912]
 [1.05106555 0.50696582 2.71146817]]
B1 = [[-0.91230853 -1.75230853 -1.50233633]
 [-1.56133665 -6.27615427 -1.56133665]
 

In [25]:
#test với ảnh mẫu
listImage = [1,2,6,7]
fileExtension = ['bmp','gif','jpg','png']
# listImage = [4]
# fileExtension = ['jpg']

# maTranAnh = [
#     img.open('images/input1.bmp').convert('RGB'),
#     img.open('images/input2.gif').convert('RGB'),
#     img.open('images/input6.jpg').convert('RGB'),
#     img.open('images/input7.png').convert('RGB'),
#     img.open('images/input4.jpg').convert('RGB'),
#     ]
A1 = taoMaTranDoiXungPerceptron(cnn.quanThe[:5])
A2 = []
for j in range(1,10):
    A2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j:5*j+5]))
B1 = taoMaTranDoiXungPerceptron(cnn.quanThe[50:55])
B2 = []
for j in range(1,10):
    B2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j+50:5*j+55]))
I = cnn.quanThe[100]
for i in range(len(listImage)):
    maTranAnh = img.open(f'images/input{listImage[i]}.{fileExtension[i]}').convert('RGB')
    m,n = maTranAnh.size
    u = np.array(maTranAnh)[:,:,0]
    print(u.shape)
    u = u/255.0*2.0-1.0
    ptvp_test = ode(hamTrangThai).set_integrator('vode').set_initial_value(u.copy().flatten(), 0.0)
    uB1 = convolve2d(u, B1, 'same')
    uB2 = tichChapBacHai(u, B2)
    ptvp_test.set_f_params(uB1, uB2, A1, A2, I, n, m)
    x = ptvp_test.integrate(cnn.t1).reshape(n, m)
    y = hamDauRa(x)
    y = np.sign(y)
    y = (y/2.0+0.5)*255
    print(f'y = {y}')
    out = img.fromarray(y).convert('RGB')
    out.save(f'images/output{listImage[i]}.png')
    print(f'done image {listImage[i]}')
    # maTranAnh.show()
    # out.show()

(256, 256)
y = [[255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 ...
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]]
done image 1
(128, 128)
y = [[255. 255.   0. ... 255. 255. 255.]
 [255. 255.   0. ... 255. 255. 255.]
 [  0.   0. 255. ... 255. 255. 255.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
done image 2
(252, 336)
y = [[255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 ...
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]]
done image 6
(316, 474)
y = [[255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255. 255.]
 ...
 [  0.   0.   0. ... 255. 255. 255.]
 [  0.   0. 255. ... 255. 255. 255.]
 [  0.   0.   0. ... 255. 2

In [20]:
#test với ảnh mẫu
# listImage = [1,2,6,7]
# fileExtension = ['bmp','gif','jpg','png']
listImage = [4]
fileExtension = ['jpg']

# maTranAnh = [
#     img.open('images/input1.bmp').convert('RGB'),
#     img.open('images/input2.gif').convert('RGB'),
#     img.open('images/input6.jpg').convert('RGB'),
#     img.open('images/input7.png').convert('RGB'),
#     img.open('images/input4.jpg').convert('RGB'),
#     ]
A1 = taoMaTranDoiXungPerceptron(cnn.quanThe[:5])
A2 = []
for j in range(1,10):
    A2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j:5*j+5]))
B1 = taoMaTranDoiXungPerceptron(cnn.quanThe[50:55])
B2 = []
for j in range(1,10):
    B2.append(taoMaTranDoiXungPerceptron(cnn.quanThe[5*j+50:5*j+55]))
I = cnn.quanThe[100]
for i in range(len(listImage)):
    maTranAnh = img.open(f'images/input{listImage[i]}.{fileExtension[i]}').convert('RGB')
    m,n = maTranAnh.size
    u = np.array(maTranAnh)[:,:,0]
    print(u.shape)
    u = u/255.0*2.0-1.0
    ptvp_test = ode(hamTrangThai).set_integrator('vode').set_initial_value(u.copy().flatten(), 0.0)
    uB1 = convolve2d(u, B1, 'same')
    uB2 = tichChapBacHai(u, B2)
    ptvp_test.set_f_params(uB1, uB2, A1, A2, I, n, m)
    x = ptvp_test.integrate(cnn.t1).reshape(n, m)
    y = hamDauRa(x)
    y = np.sign(y)
    y = (y/2.0+0.5)*255
    print(f'y = {y}')
    out = img.fromarray(y).convert('RGB')
    out.save(f'images/output{listImage[i]}.png')
    print(f'done image {listImage[i]}')
    # maTranAnh.show()
    # out.show()

(795, 1080)
y = [[255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 [255. 255. 255. ...   0.   0.   0.]
 ...
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
done image 4
